<a href="https://colab.research.google.com/github/Troublem1/Portable_Water_Classification/blob/main/003PW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q -r /content/drive/MyDrive/workSpace/Potable_Water/configs/PW_requirements.txt
# !pip install -q optuna==2.9.1

In [5]:
from omegaconf import OmegaConf
from zindi.user import Zindian
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import shutil

import catboost as cb
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
# import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc,roc_auc_score, log_loss, f1_score
from sklearn import preprocessing
import seaborn as sns
import joblib
import matplotlib.pyplot as plt
from google.colab import files
import time
# import tensorflow as tf

In [6]:
os.makedirs('/content/drive/MyDrive/workSpace/Potable_Water/configs/' , exist_ok=True )

In [7]:
params = {'defaults':{'seed_value':2021, 'nKfold':10},
'dir': {'model_name':'001',
        'data_path':'/content/drive/MyDrive/workSpace/Potable_Water/data/',
        'train_file':'/content/drive/MyDrive/workSpace/Potable_Water/data/Train.csv' ,
        'test_file':'/content/drive/MyDrive/workSpace/Potable_Water/data/Test.csv' ,
        'submit_file':'/content/drive/MyDrive/workSpace/Potable_Water/data/SampleSubmission.csv',
        'model_path':'/content/drive/MyDrive/workSpace/Potable_Water/models/',
        'checkpoints_path':'/content/drive/MyDrive/workSpace/Potable_Water/checkpoints/',
        'predictions_path':'/content/drive/MyDrive/workSpace/Potable_Water/submissions/'},

# 'preprocessing':{'remove_traindf_cols':['user_id','CHURN'], 
#                'remove_testdf_cols':['user_id'],'cat_cols_remove':'user_id',
#                'feature_cols':['FREQUENCE_RECH','DATA_VOLUME', 'TIGO', 'FREQ_TOP_PACK', 'ARPU_SEGMENT', 'ORANGE', 
#                                'ON_NET', 'REVENUE', 'MONTANT', 'REGULARITY', 'FREQUENCE', 'ZONE2', 'ZONE1'],
#                'category_cols':['REGION', 'TENURE', 'MRG', 'TOP_PACK'],
#                'target_col':'CHURN'},
          
  'model':{'boosting_type': 'gbdt','path_smooth':0.3 ,'min_data_in_leaf':1000,
          #  'class_weights':'balanced',
          #  'is_unbalance':True,
          #  'num_class':1,
           'learning_rate':0.2,'num_leaves':30 ,'objective': 'binary','metric': 'auc','max_depth':20,
                'cat_smooth':20,'n_estimators': 98,'colsample_bytree' : 1 ,'deterministic':True,
                # 'categorical_feature':['REGION', 'TENURE', 'MRG', 'TOP_PACK'],
                'seed': 2021,'silent':False,'num_iterations':900 ,'reg_lambda':2,'reg_alpha':2}}

if '001PW.yaml' in os.listdir('/content/drive/MyDrive/workSpace/Potable_Water/configs/'):
  config_ = OmegaConf.load('/content/drive/MyDrive/workSpace/Potable_Water/configs/001PW.yaml')
else:
  OmegaConf.save(config=params, f='/content/drive/MyDrive/workSpace/Potable_Water/configs/001PW.yaml')
  config_ = OmegaConf.load('/content/drive/MyDrive/workSpace/Potable_Water/configs/001PW.yaml')


In [8]:
print(OmegaConf.to_yaml(config_.dir))

model_name: '001'
data_path: /content/drive/MyDrive/workSpace/Potable_Water/data/
train_file: /content/drive/MyDrive/workSpace/Potable_Water/data/Train.csv
test_file: /content/drive/MyDrive/workSpace/Potable_Water/data/Test.csv
submit_file: /content/drive/MyDrive/workSpace/Potable_Water/data/SampleSubmission.csv
model_path: /content/drive/MyDrive/workSpace/Potable_Water/models/
checkpoints_path: /content/drive/MyDrive/workSpace/Potable_Water/checkpoints/
predictions_path: /content/drive/MyDrive/workSpace/Potable_Water/submissions/



In [9]:
os.makedirs(config_.dir.model_path , exist_ok=True )
os.makedirs(config_.dir.predictions_path, exist_ok=True)

In [10]:
# # create a user object
# my_username = "ZzyZx"
# user = Zindian(username = my_username)

# # Select a Zindi challenge
# user.select_a_challenge()  #current index 5
# user.which_challenge

# # Download the dataset of the selected challenge for the sfc select 2
# user.download_dataset(destination=config_.dir.data_path)

In [11]:
print(OmegaConf.to_yaml(config_.defaults))

seed_value: 2021
nKfold: 10



In [12]:
# Setting SEED to Reproduce Same Results even with "GPU"
import os
os.environ['PYTHONHASHSEED'] = str(config_.defaults.seed_value)
import random
random.seed(config_.defaults.seed_value)
import numpy as np
np.random.seed(config_.defaults.seed_value)


In [13]:
print(OmegaConf.to_yaml(config_.dir))

model_name: '001'
data_path: /content/drive/MyDrive/workSpace/Potable_Water/data/
train_file: /content/drive/MyDrive/workSpace/Potable_Water/data/Train.csv
test_file: /content/drive/MyDrive/workSpace/Potable_Water/data/Test.csv
submit_file: /content/drive/MyDrive/workSpace/Potable_Water/data/SampleSubmission.csv
model_path: /content/drive/MyDrive/workSpace/Potable_Water/models/
checkpoints_path: /content/drive/MyDrive/workSpace/Potable_Water/checkpoints/
predictions_path: /content/drive/MyDrive/workSpace/Potable_Water/submissions/



In [14]:
train_df = pd.read_csv(config_.dir.train_file)
test_df = pd.read_csv(config_.dir.test_file)
sub_df = pd.read_csv(config_.dir.submit_file)

In [15]:
train_df.head()

,region_area_,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,region_area_1,3.716080,180.196811,18630.362669,6.330435,NaN,602.990359,15.285013,56.444076,4.500656,0
1,region_area_100,4.270716,217.611599,24405.555346,7.086700,297.951852,359.623910,13.583526,54.589840,3.763906,0
2,region_area_1000,6.648849,232.462013,30733.808648,5.656462,344.670335,267.809179,17.863444,35.411181,4.827527,0
3,region_area_1001,NaN,261.753637,21174.713291,7.314074,NaN,582.848407,15.215849,90.367455,4.217553,0
4,region_area_1002,9.323584,263.484560,15751.480111,5.196259,297.593625,409.878704,10.727265,74.669285,3.006650,0


In [16]:
test_df.head()

,region_area_,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
0,region_area_0,NaN,255.664346,20791.623791,6.995401,369.261555,574.413654,10.484783,87.105970,2.963135
1,region_area_10,7.360640,216.294688,32452.919220,7.245890,327.369467,435.488419,15.691810,78.855016,3.662292
2,region_area_1006,7.230845,228.348694,17864.994196,5.513418,344.338446,377.794992,20.017950,59.418053,5.622018
3,region_area_1010,4.797460,241.089018,21280.551369,5.883692,327.539307,367.541193,11.818821,63.978068,4.038503
4,region_area_1011,9.319176,250.107456,14294.545161,7.480130,366.796865,348.546450,15.320293,63.506468,4.129230


In [17]:
sub_df.head()

,region_area_,Potability
0,region_area_0,1
1,region_area_10,1
2,region_area_1006,1
3,region_area_1010,1
4,region_area_1011,1


In [ ]:
# #check distribution of customer in each country across location_type and bank_account columns
# pd.crosstab(train_df.REGION,train_df.CHURN)

Variable Descriptions

- pH value: PH is an important parameter in evaluating the acid–base balance of water. It is also the indicator of acidic or alkaline condition of water status. WHO has recommended a maximum permissible limit of pH from 6.5 to 8.5. The current investigation ranges were 6.52–6.83 which are in the range of WHO standards.
- Hardness: Hardness is mainly caused by calcium and magnesium salts. These salts are dissolved from geologic deposits through which water travels. The length of time water is in contact with hardness producing material helps determine how much hardness there is in raw water. Hardness was originally defined as the capacity of water to precipitate soap caused by Calcium and Magnesium.
- Solids (Total dissolved solids - TDS): Water has the ability to dissolve a wide range of inorganic and some organic minerals or salts such as potassium, calcium, sodium, bicarbonates, chlorides, magnesium, sulfates etc. These minerals produced an unwanted taste and diluted color in the appearance of water. This is the important parameter for the use of water. The water with high TDS value indicates that water is highly mineralized. The desired limit for TDS is 500 mg/l and maximum limit is 1000 mg/l which is prescribed for drinking purposes.
- Chloramines: Chlorine and chloramine are the major disinfectants used in public water systems. Chloramines are most commonly formed when ammonia is added to chlorine to treat drinking water. Chlorine levels up to 4 milligrams per liter (mg/L or 4 parts per million (ppm)) are considered safe in drinking water.
- Sulfate: Sulfates are naturally occurring substances that are found in minerals, soil, and rocks. They are present in ambient air, groundwater, plants, and food. The principal commercial use of sulfate is in the chemical industry. Sulfate concentration in seawater is about 2,700 milligrams per liter (mg/L). It ranges from 3 to 30 mg/L in most freshwater supplies, although much higher concentrations (1000 mg/L) are found in some geographic locations.
- Conductivity: Pure water is not a good conductor of electric current rather it's a good insulator. Increase in ions concentration enhances the electrical conductivity of water. Generally, the amount of dissolved solids in water determines the electrical conductivity. Electrical conductivity (EC) actually measures the ionic process of a solution that enables it to transmit current. According to WHO standards, EC value should not exceed 400 μS/cm.
- Organic_carbon: Total Organic Carbon (TOC) in source waters comes from decaying natural organic matter (NOM) as well as synthetic sources. TOC is a measure of the total amount of carbon in organic compounds in pure water. According to the US EPA < 2 mg/L as TOC in treated / drinking water, and < 4 mg/Lit in source water which is used for treatment.
- Trihalomethanes: THMs are chemicals which may be found in water treated with chlorine. The concentration of THMs in drinking water varies according to the level of organic material in the water, the amount of chlorine required to treat the water, and the temperature of the water that is being treated. THM levels up to 80 ppm is considered safe in drinking water.
- Turbidity: The turbidity of water depends on the quantity of solid matter present in the suspended state. It is a measure of light emitting properties of water and the test is used to indicate the quality of waste discharge with respect to colloidal matter. The mean turbidity value obtained for Wondo Genet Campus (0.98 NTU) is lower than the WHO recommended value of 5.00 NTU.
- Potability: Indicates if water is safe for human consumption where 1 means Potable and 0 means Not potable.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

### MY GBM BASELINES 

- XGBOOST

In [ ]:
# # Baseline
# """XGBOOST on GPU"""
# from sklearn.model_selection import KFold
# import xgboost as xgb
# # from sklearn.metrics import mean_squared_error

# X = train_df.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
# y = train_df['Potability'] #[config_.preprocess.target[0]].astype(int)

# test_new = test_df.drop(columns=['region_area_']) #config_.preprocess.to_drop)
# test_fields = test_df['region_area_'] #[config_.preprocess.to_drop[0]]  #field

# kf = KFold(n_splits = 5,shuffle=True,random_state=2021)
# feats = pd.DataFrame({'features': X.columns}) 
# xgb_predictions = []
# cv_score_ = 0
# oof_preds = np.zeros((X.shape[0],))


# for i,(tr_index,test_index) in enumerate(kf.split(X,y)):
#     #   print()
#     print(f'######### FOLD {i+1} / {kf.n_splits} ')
  
#     X_train,y_train = X.iloc[tr_index,:],y[tr_index]
#     X_test,y_test = X.iloc[test_index,:],y[test_index]
#       # merror or mlogloss
#     gbm = xgb.XGBClassifier(eval_metric = 'logloss',
#                             # objective='multi:softprob',#tree_method='gpu_hist',
#                               grow_policy='lossguide', 
#                               # gpu_id=0, 
#                             n_estimators = 300, scale_pos_weight=30,
#                               learning_rate = 0.03, random_state = 2021, colsample_bytree=1 ,reg_alpha=1, reg_lambda=1)

#     gbm.fit(X_train,y_train,eval_set = [(X_test, y_test)],early_stopping_rounds  = 50, verbose=200)

#     cv_score_ += f1_score(y_test, gbm.predict(X_test)) / kf.n_splits
#     oof_preds[test_index] = gbm.predict_proba(X_test)[:,1]

#     preds = gbm.predict_proba(test_new[X_train.columns] , gbm.best_iteration)[:,1]
#     xgb_predictions.append(preds)

#     feats[f'Fold {i}'] = gbm.feature_importances_

# feats['Importances'] = feats.mean(axis=1)
# print( ' CV F1_SCORE : ',cv_score_)
# # preds_xgb = np.average(gbm_predictions, axis=0)
# # print(preds_xgb.shape

In [ ]:
# xgbpreds_mean = np.mean(xgb_predictions, axis=0)# mean of all the predictions
# final = [] #list to store the final results 
# for x in xgbpreds_mean:
#     if x >= 0.51:
#         final.append(1)
#     else:
#         final.append(0)
# submit = test_df.copy() #Copy the submission file to the variable submit 
# submit['Potability'] = final
# submit[['region_area_','Potability']].to_csv('000xgbbaseline.csv',index=False) #creating and naming the file ready for submission

In [ ]:
# submit[['region_area_','Potability']].head()

In [ ]:
# seed = 2021 # seed
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed) # for cross validation
# sklearnscores = []
# # torchscores = []
# catpreds= []


# np.random.seed(2021)
# seeds = np.random.randint(low=1, high=3000, size=5)

# X = train_df.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
# y = train_df['Potability'] #[config_.preprocess.target[0]].astype(int)

# test_new = test_df.drop(columns=['region_area_']) #config_.preprocess.to_drop)
# test_fields = test_df['region_area_'] #[config_.preprocess.to_drop[0]]  #field

# #creating a for loop for the stratified k fold
# i = 0
# for train, test in skf.split(X, y):
#     x_train, x_test, y_train, y_test = X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test]# spliting the data


#     estimator = cb.CatBoostClassifier(iterations=10000,                     #has_time=True ,bootstrap_type='No',random_strength=0,
#                                    learning_rate=0.08,
#                                   #  objective='MultiClass',
#                                   use_best_model=True,
#                                    reg_lambda=1,
#                                    random_seed=seed,         #depth=3,border_count=32, l2_leaf_reg=3,
#                                   #  task_type='GPU',
#                                   #  loss_function='MultiClass',
#                                   early_stopping_rounds=50)

#     # estimator = CatBoostClassifier(**params)
#     # parameters for the algorithm this was done by manual tuning

#     estimator.fit( x_train, y_train, eval_set = (x_test,y_test),verbose=100 ,early_stopping_rounds=50)# fitting on train data
    
#     print('Number of splits trained {} '.format(i+1))
#     score1 =  f1_score(y_test, gbm.predict(x_test))  # checking the cross_entropy loss

#     # score2 = torchLoss(estimator.predict_proba(x_test) , y_test)
#     # cm = ConfusionMatrix(actual_vector=np.array(y_test), predict_vector=estimator.predict(x_test))
#     # plt.show(cm.print_matrix())
#     print('f1_score score: {}'.format(score1))

#     catpred = estimator.predict_proba(test_new)[:,1] # making prediction probabities
#     sklearnscores.append(score1)
#     # torchscores.append(score2)

#     catpreds.append(catpred)
#     i += 1
# print('mean f1_scores: {} '.format(np.mean(sklearnscores)))

In [ ]:
# catpreds_mean = np.mean(catpreds, axis=0)# mean of all the predictions
# final = [] #list to store the final results 
# for x in catpreds_mean:
#     if x >= 0.51:
#         final.append(1)
#     else:
#         final.append(0)
# submit = test_df.copy() #Copy the submission file to the variable submit 
# submit['Potability'] = final
# submit[['region_area_','Potability']].to_csv('000catbaseline.csv',index=False) #creating and naming the file ready for submission

In [ ]:
# submit[['region_area_','Potability']].head()

LGBM

In [ ]:
# since = time.time()
# skf = StratifiedKFold(n_splits=config_.defaults.nKfold, shuffle=True, random_state=config_.defaults.seed_value) # for cross validation
# sklearnscores = []
# lgbmpreds= []

# np.random.seed(config_.defaults.seed_value)
# seeds = np.random.randint(low=1, high=3000, size=10)

# X = train_df.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
# y = train_df['Potability'] #[config_.preprocess.target[0]].astype(int)

# test_new = test_df.drop(columns=['region_area_']) #config_.preprocess.to_drop)
# test_fields = test_df['region_area_'] #[config_.preprocess.to_drop[0]]  #field

# #creating a for loop for the stratified k fold
# i = 0
# for train, test in skf.split(X, y):
#     print(f'########### Fold {i+1} / {skf.n_splits} ')
    
#     x_train, x_test, y_train, y_test = X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test]# spliting the data
    
#     estimator = lgb.LGBMClassifier( boosting_type='gbdt', num_leaves=30,scale_pos_weight=2, num_class =1,
#                                max_depth=- 1, learning_rate=0.08, n_estimators=800, subsample_for_bin=300000,
#                                deterministic=True,objective='binary', class_weight=None, min_split_gain=0.0,
#                                min_child_weight=0.001, min_child_samples=20, subsample=1.0, subsample_freq=0, 
#                                colsample_bytree=1.0, reg_alpha=1, reg_lambda=1, random_state=seeds[i], n_jobs=- 1,
#                                silent=True, device_type='CPU',importance_type='split')

# #     estimator = LGBMClassifier(**config_.model,random_state=seeds[i])
# # early_stopping_rounds=50, 
#     estimator.fit( x_train, y_train, eval_set = (x_test,y_test),eval_metric = 'binary_error',  verbose=100, early_stopping_rounds=50 )# fitting on train data

#     score1 = f1_score(y_test, estimator.predict(x_test)) # checking the cross_entropy loss
#     print('f1 score: {}'.format(score1))

#     pred = estimator.predict_proba(test_new)[:,1] # making prediction probabities
#     sklearnscores.append(score1)

#     lgbmpreds.append(pred)
#     i += 1

# print('mean f1_scores from sklearn: {}'.format(np.mean(sklearnscores)))

# time_elapsed = time.time() - since
# print(f'Pre-process the data: {time_elapsed // 60} m {time_elapsed % 60} s')

In [ ]:
# lgbmpreds_mean = np.mean(lgbmpreds, axis=0)# mean of all the predictions
# final = [] #list to store the final results 
# for x in lgbmpreds_mean:
#     if x >= 0.51:
#         final.append(1)
#     else:
#         final.append(0)
# submit = test_df.copy() #Copy the submission file to the variable submit 
# submit['Potability'] = final
# submit[['region_area_','Potability']].to_csv('000lgbmbaseline.csv',index=False) #creating and naming the file ready for submission

### XVII BASELINE

In [ ]:
# #Here the cross validation by using the StratifiedKfold used to split the data into train,test and validation and repeatedly and Using the LGBMClassifier as Algorithm for building the model 
# skf =StratifiedKFold(n_splits=10, shuffle=True,random_state=2021) # for cross validation by StratifiedKfold
# scores = [] #list to store the sores obtained in training the model 
# preds= [] #list to store the predictions 

# X = train_df.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
# y = train_df['Potability'] #[config_.preprocess.target[0]].astype(int)

# test_new = test_df.drop(columns=['region_area_']) #config_.preprocess.to_drop)
# test_fields = test_df['region_area_'] #[config_.preprocess.to_drop[0]]  #field

# #creating a for loop for the stratified k fold to perform the cross validation 
# i = 1
# for train, test in skf.split(X, y):
#     x_train, x_test= X.iloc[train], X.iloc[test] #spliting the data for x 
#     y_train, y_test = y.iloc[train],y.iloc[test] #spliting the data for y 
#     model = lgb.LGBMClassifier(learning_rate=0.08, n_estimators = 96,cat_smooth=10,metrics='binary_error',
#                                scale_pos_weight= 3.6,max_depth=16, seed =2021,
#                                num_leaves=49,reg_lambda=0.3) #Creating the model and tuning it with the different parameter 
#     model.fit(x_train, y_train)# fitting  the model on train data
#     print(i,'Split Trained') #print the splits that trained 
#     score = f1_score(y_test,model.predict(x_test)) # checking the accuracy of the model 
#     pred = model.predict_proba(test_new)[:,1] # making prediction
#     scores.append(score)
#     preds.append(pred)
#     i += 1
# print('Total Mean Score Obtained :',np.mean(scores))

# preds_mean = np.mean(preds, axis=0)# mean of all the predictions
# final = [] #list to store the final results 
# for x in preds_mean:
#     if x >= 0.51:
#         final.append(1)
#     else:
#         final.append(0)

# submit = test_df.copy() #Copy the submission file to the variable submit 
# submit['Potability'] = final
# submit[['region_area_','Potability']].to_csv('000xviibaseline.csv',index=False) #creating and naming the file ready for submission

In [ ]:
# 0.5389573339467848 # LB: 0.56

# OPTUNA TRIAL

In [ ]:
# # create trial function
# OPTUNA_OPTIMIZATION = True

# def objective(trial):
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1)
    
# #  boosting_type='gbdt', =30,scale_pos_weight=1,  =9,
# #    max_depth=-1, learning_rate=0.08, n_estimators=2000, subsample_for_bin=300000,
# #   deterministic=True,objective='logloss', class_weight=None, min_split_gain=0.0,
# #   min_child_weight=0.001, min_child_samples=20, =0.80, subsample_freq=20, 
# #   colsample_bytree=0.30, reg_alpha=1, reg_lambda=1, random_state=seeds[i], n_jobs=- 1,
# #    silent=True, =
#     params = {
#         'n_estimators':trial.suggest_int("n_estimators", 90, 700),
#         # 'objective': trial.suggest_categorical('objective', ['binary','cross_entropy']),
#         'importance_type':trial.suggest_categorical('importance_type', ['gain','split']),
#         'num_leaves':trial.suggest_int('num_leaves', 10, 100),
#         'learning_rate' : trial.suggest_uniform('learning_rate',0.02,1),
#         'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
#         'reg_alpha': trial.suggest_uniform('reg_alpha',1e-5,100),
#         'colsample_bytree': trial.suggest_uniform('colsample_bytree',0.1,1),
#         'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 3, 20),
#         'max_depth': trial.suggest_int('depth',1,15),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',20,300),
# #         'bagging_freq': trial.suggest_int('bagging_freq',0,10),
#         'subsample': trial.suggest_uniform('subsample', 0.1, 1),
#         'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1),
#         'min_sum_hessian_in_leaf': trial.suggest_uniform('min_sum_hessian_in_leaf', 1e-3, 1),
# #         'max_bin': trial.suggest_int('max_bin',100,500),
#         'random_state': 2021, 
#         'num_class':  1, 
#         # "device": "gpu",
#         # "gpu_platform_id": 0,
#         # "gpu_device_id": 0,
# #         'verbose': False,
# #         'task_type' : 'GPU',
# #         'devices' : '0'
#     }
    
#     # if params['bootstrap_type'] == 'Bayesian':
#     #     params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 10)
#     # elif params['bootstrap_type'] == 'Bernoulli':
#     #     params['subsample'] = trial.suggest_float('subsample', 0.1, 1)
    
#     model = lgb.LGBMClassifier(**params)
#     model.fit(
#         X_train, y_train,
#         eval_set = (X_test,y_test),
#         early_stopping_rounds=200,verbose=200 
# #         use_best_model=True
#     )

#     # # validation AUC prediction
#     # y_hat = model.predict_proba(X_test)[:,1]
#     # fpr, tpr, _ = roc_curve(y_test, y_hat)
#     # score = auc(fpr, tpr)
    

#     # validation log loss prediction
#     # y_hat = model.predict_proba(X_test)
#     # score = log_loss(y_test, y_hat)
    
#     # validation F1 predictions
#     y_hat = model.predict(X_test)
#     score = f1_score(y_test, y_hat)
    
#     return score

In [ ]:
# X = train_df.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
# y = train_df['Potability'] #[config_.preprocess.target[0]].astype(int)

In [ ]:
# #create optuna study
# study = optuna.create_study(
#     direction='maximize',
#     study_name='LGBMClf'
# )

# study.optimize(
#     objective, 
#     n_trials=300
# )

[I 2021-09-22 19:05:36,975] A new study created in memory with name: LGBMClf
[I 2021-09-22 19:05:37,050] Trial 0 finished with value: 0.0 and parameters: {'n_estimators': 444, 'importance_type': 'split', 'num_leaves': 58, 'learning_rate': 0.590368855420905, 'reg_lambda': 10.791273756853146, 'reg_alpha': 37.3363318607632, 'colsample_bytree': 0.8871093424675291, 'scale_pos_weight': 9.224693284041201, 'depth': 4, 'min_data_in_leaf': 109, 'subsample': 0.8523402501226346, 'bagging_fraction': 0.6331754267919684, 'min_sum_hessian_in_leaf': 0.22223963966012944}. Best is trial 0 with value: 0.0.
[I 2021-09-22 19:05:37,111] Trial 1 finished with value: 0.0 and parameters: {'n_estimators': 368, 'importance_type': 'split', 'num_leaves': 32, 'learning_rate': 0.6437316684321902, 'reg_lambda': 55.17740066049302, 'reg_alpha': 29.940546916039143, 'colsample_bytree': 0.7298965781873297, 'scale_pos_weight': 5.074531600595723, 'depth': 10, 'min_data_in_leaf': 235, 'subsample': 0.8467422988990422, 'bagging

[LightGBM] [Warning] min_data_in_leaf is set=109, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=109
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.22223963966012944, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.22223963966012944
[LightGBM] [Warning] bagging_fraction is set=0.6331754267919684, subsample=0.8523402501226346 will be ignored. Current value: bagging_fraction=0.6331754267919684
Training until validation scores don't improve for 200 rounds
[200]	valid_0's binary_logloss: 0.702674
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.702674
[LightGBM] [Warning] min_data_in_leaf is set=235, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=235
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.10225776149212226, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.10225776149212226
[LightGBM] [Warning] bagging_fraction is set=0.22026853999108487, sub

[I 2021-09-22 19:05:37,239] Trial 3 finished with value: 0.0 and parameters: {'n_estimators': 401, 'importance_type': 'gain', 'num_leaves': 58, 'learning_rate': 0.8356199273633377, 'reg_lambda': 26.48487761519524, 'reg_alpha': 21.11949780485782, 'colsample_bytree': 0.7201120447280136, 'scale_pos_weight': 12.10499566870847, 'depth': 11, 'min_data_in_leaf': 32, 'subsample': 0.9697866116201368, 'bagging_fraction': 0.24087615321276992, 'min_sum_hessian_in_leaf': 0.4360171703434167}. Best is trial 0 with value: 0.0.
[I 2021-09-22 19:05:37,287] Trial 4 finished with value: 0.0 and parameters: {'n_estimators': 133, 'importance_type': 'gain', 'num_leaves': 60, 'learning_rate': 0.31612926781697726, 'reg_lambda': 47.26662508280218, 'reg_alpha': 78.30336833715509, 'colsample_bytree': 0.7740646072691456, 'scale_pos_weight': 12.820845159366629, 'depth': 5, 'min_data_in_leaf': 243, 'subsample': 0.9704912516165524, 'bagging_fraction': 0.16915061569450257, 'min_sum_hessian_in_leaf': 0.9241200972104899

[200]	valid_0's binary_logloss: 0.702674
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.702674
[LightGBM] [Warning] min_data_in_leaf is set=243, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=243
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.9241200972104899, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.9241200972104899
[LightGBM] [Warning] bagging_fraction is set=0.16915061569450257, subsample=0.9704912516165524 will be ignored. Current value: bagging_fraction=0.16915061569450257
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[1]	valid_0's binary_logloss: 0.702674
[LightGBM] [Warning] min_data_in_leaf is set=250, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=250
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.29660177537859095, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.

[I 2021-09-22 19:05:37,472] Trial 7 finished with value: 0.0 and parameters: {'n_estimators': 155, 'importance_type': 'gain', 'num_leaves': 51, 'learning_rate': 0.13115178870396485, 'reg_lambda': 22.69737330612657, 'reg_alpha': 60.85094492700962, 'colsample_bytree': 0.2646908236187067, 'scale_pos_weight': 13.81778148014418, 'depth': 12, 'min_data_in_leaf': 296, 'subsample': 0.3721935610026358, 'bagging_fraction': 0.13341946496357177, 'min_sum_hessian_in_leaf': 0.5369395366388487}. Best is trial 0 with value: 0.0.
[I 2021-09-22 19:05:37,533] Trial 8 finished with value: 0.0 and parameters: {'n_estimators': 446, 'importance_type': 'split', 'num_leaves': 21, 'learning_rate': 0.39403626122175234, 'reg_lambda': 18.525733268298715, 'reg_alpha': 49.35304769960683, 'colsample_bytree': 0.9216770224611075, 'scale_pos_weight': 13.196295397857854, 'depth': 15, 'min_data_in_leaf': 152, 'subsample': 0.8104265094028137, 'bagging_fraction': 0.1735776439625611, 'min_sum_hessian_in_leaf': 0.899980812216

...

[LightGBM] [Warning] min_data_in_leaf is set=292, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=292
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.09159108004930447, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.09159108004930447
[LightGBM] [Warning] bagging_fraction is set=0.4720404242060455, subsample=0.14972165468823573 will be ignored. Current value: bagging_fraction=0.4720404242060455
Training until validation scores don't improve for 200 rounds
[200]	valid_0's binary_logloss: 0.702674
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.702674
[LightGBM] [Warning] min_data_in_leaf is set=55, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=55
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.015863534539331724, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.015863534539331724
[LightGBM] [Warning] bagging_fraction is set=0.7374918898308622, sub

[I 2021-09-22 19:06:03,274] Trial 297 finished with value: 0.569620253164557 and parameters: {'n_estimators': 243, 'importance_type': 'split', 'num_leaves': 58, 'learning_rate': 0.4787651443868317, 'reg_lambda': 20.05651014565372, 'reg_alpha': 51.431631785985054, 'colsample_bytree': 0.28348632367850146, 'scale_pos_weight': 10.735662497674802, 'depth': 1, 'min_data_in_leaf': 56, 'subsample': 0.3212386505585786, 'bagging_fraction': 0.734790622341816, 'min_sum_hessian_in_leaf': 0.0017314032025078606}. Best is trial 14 with value: 0.569620253164557.
[I 2021-09-22 19:06:03,385] Trial 298 finished with value: 0.569620253164557 and parameters: {'n_estimators': 271, 'importance_type': 'split', 'num_leaves': 54, 'learning_rate': 0.5264194013622541, 'reg_lambda': 20.0161934982498, 'reg_alpha': 51.051841395035154, 'colsample_bytree': 0.32584586357699624, 'scale_pos_weight': 10.789525336230222, 'depth': 1, 'min_data_in_leaf': 57, 'subsample': 0.3164268712394581, 'bagging_fraction': 0.7221074966169

[LightGBM] [Warning] min_data_in_leaf is set=56, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=56
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.0017314032025078606, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.0017314032025078606
[LightGBM] [Warning] bagging_fraction is set=0.734790622341816, subsample=0.3212386505585786 will be ignored. Current value: bagging_fraction=0.734790622341816
Training until validation scores don't improve for 200 rounds
[200]	valid_0's binary_logloss: 1.27291
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.773693
[LightGBM] [Warning] min_data_in_leaf is set=57, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=57
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.019210537880167834, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.019210537880167834
[LightGBM] [Warning] bagging_fraction is set=0.7221074966169765, subsa

[I 2021-09-22 19:06:03,504] Trial 299 finished with value: 0.569620253164557 and parameters: {'n_estimators': 252, 'importance_type': 'split', 'num_leaves': 36, 'learning_rate': 0.6309004964103724, 'reg_lambda': 11.240585885144107, 'reg_alpha': 25.32833594818119, 'colsample_bytree': 0.40768438905641874, 'scale_pos_weight': 11.537013691459789, 'depth': 5, 'min_data_in_leaf': 44, 'subsample': 0.12013962111261825, 'bagging_fraction': 0.5450391213851564, 'min_sum_hessian_in_leaf': 0.08056808655353781}. Best is trial 14 with value: 0.569620253164557.


[LightGBM] [Warning] min_data_in_leaf is set=44, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=44
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.08056808655353781, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.08056808655353781
[LightGBM] [Warning] bagging_fraction is set=0.5450391213851564, subsample=0.12013962111261825 will be ignored. Current value: bagging_fraction=0.5450391213851564
Training until validation scores don't improve for 200 rounds
[200]	valid_0's binary_logloss: 1.21664
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.862565


In [ ]:
# print(f"Best Trial: {study.best_trial.value}")
# print(f"Best Params: {study.best_trial.params}")

Best Trial: 0.569620253164557
Best Params: {'n_estimators': 548, 'importance_type': 'split', 'num_leaves': 71, 'learning_rate': 0.5451820431683637, 'reg_lambda': 0.18982147693285611, 'reg_alpha': 65.58618060010909, 'colsample_bytree': 0.5718965752147983, 'scale_pos_weight': 9.722381028559631, 'depth': 9, 'min_data_in_leaf': 66, 'subsample': 0.8780495139659923, 'bagging_fraction': 0.37946033709168714, 'min_sum_hessian_in_leaf': 0.11449230117536044}


In [ ]:
# H_params = study.best_trial.params

In [ ]:
# H_params = {'n_estimators': 586, 'objective': 'binary', 'importance_type': 'gain', 'num_leaves': 82, 'learning_rate': 0.41490608340120194, 'reg_lambda': 5.708408942536359, 'reg_alpha': 5.517693129732991, 'colsample_bytree': 0.84460849025194, 'scale_pos_weight': 3.6213684221040014, 'max_depth': 14, 'min_data_in_leaf': 56, 'subsample': 0.28937062760787574, 'bagging_fraction': 0.3376651043328511, 'min_sum_hessian_in_leaf': 0.9208910356602873}


In [ ]:
# #Here the cross validation by using the StratifiedKfold used to split the data into train,test and validation and repeatedly and Using the LGBMClassifier as Algorithm for building the model 
# skf =StratifiedKFold(n_splits=10, shuffle=True,random_state=2021) # for cross validation by StratifiedKfold
# scores = [] #list to store the sores obtained in training the model 
# preds= [] #list to store the predictions 

# X = train_df.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
# y = train_df['Potability'] #[config_.preprocess.target[0]].astype(int)

# test_new = test_df.drop(columns=['region_area_']) #config_.preprocess.to_drop)
# test_fields = test_df['region_area_'] #[config_.preprocess.to_drop[0]]  #field

# #creating a for loop for the stratified k fold to perform the cross validation 
# i = 1
# for train, test in skf.split(X, y):
#     x_train, x_test= X.iloc[train], X.iloc[test] #spliting the data for x 
#     y_train, y_test = y.iloc[train],y.iloc[test] #spliting the data for y 
#     model = lgb.LGBMClassifier(**H_params) #Creating the model and tuning it with the different parameter 
#     model.fit(x_train, y_train)# fitting  the model on train data
#     print(i,'Split Trained') #print the splits that trained 
#     score = f1_score(y_test,model.predict(x_test)) # checking the accuracy of the model 
#     pred = model.predict_proba(test_new)[:,1] # making prediction
#     scores.append(score)
#     preds.append(pred)
#     i += 1
# print('Total Mean Score Obtained :',np.mean(scores))

# preds_mean = np.mean(preds, axis=0)# mean of all the predictions
# final = [] #list to store the final results 
# for x in preds_mean:
#     if x >= 0.51:
#         final.append(1)
#     else:
#         final.append(0)

# submit = test_df.copy() #Copy the submission file to the variable submit 
# submit['Potability'] = final
# submit[['region_area_','Potability']].to_csv('001xviibaseline.csv',index=False) #creating and naming the file ready for submission

[LightGBM] [Warning] bagging_fraction is set=0.3376651043328511, subsample=0.28937062760787574 will be ignored. Current value: bagging_fraction=0.3376651043328511
[LightGBM] [Warning] min_data_in_leaf is set=56, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=56
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.9208910356602873, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.9208910356602873
1 Split Trained
[LightGBM] [Warning] bagging_fraction is set=0.3376651043328511, subsample=0.28937062760787574 will be ignored. Current value: bagging_fraction=0.3376651043328511
[LightGBM] [Warning] min_data_in_leaf is set=56, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=56
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.9208910356602873, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.9208910356602873
2 Split Trained
[LightGBM] [Warning] bagging_fraction is set=0.33766510433

In [ ]:
# #Here the cross validation by using the StratifiedKfold used to split the data into train,test and validation and repeatedly and Using the LGBMClassifier as Algorithm for building the model 
# skf =StratifiedKFold(n_splits=10, shuffle=True,random_state=2021) # for cross validation by StratifiedKfold
# scores = [] #list to store the sores obtained in training the model 
# preds= [] #list to store the predictions 

# X = train_df.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
# y = train_df['Potability'] #[config_.preprocess.target[0]].astype(int)

# test_new = test_df.drop(columns=['region_area_']) #config_.preprocess.to_drop)
# test_fields = test_df['region_area_'] #[config_.preprocess.to_drop[0]]  #field

# #creating a for loop for the stratified k fold to perform the cross validation 
# i = 1
# for train, test in skf.split(X, y):
#     x_train, x_test= X.iloc[train], X.iloc[test] #spliting the data for x 
#     y_train, y_test = y.iloc[train],y.iloc[test] #spliting the data for y 
#     model = lgb.LGBMClassifier(learning_rate=0.08, n_estimators = 96,cat_smooth=10,metrics='binary_error',
#                                scale_pos_weight= 3.6,max_depth=16, seed =2021,
#                                num_leaves=49,reg_lambda=0.3) #Creating the model and tuning it with the different parameter 
#     model.fit(x_train, y_train)# fitting  the model on train data
#     print(i,'Split Trained') #print the splits that trained 
#     score = f1_score(y_test,model.predict(x_test)) # checking the accuracy of the model 
#     pred = model.predict_proba(test_new)[:,1] # making prediction
#     scores.append(score)
#     preds.append(pred)
#     i += 1
# print('Total Mean Score Obtained :',np.mean(scores))

# preds_mean = np.mean(preds, axis=0)*0.5 + preds_mean*0.5# mean of all the predictions
# final = [] #list to store the final results 
# for x in preds_mean:
#     if x >= 0.51:
#         final.append(1)
#     else:
#         final.append(0)

# submit = test_df.copy() #Copy the submission file to the variable submit 
# submit['Potability'] = final
# submit[['region_area_','Potability']].to_csv('001xviibaseline2.csv',index=False) #creating and naming the file ready for submission

1 Split Trained
2 Split Trained
3 Split Trained
4 Split Trained
5 Split Trained
6 Split Trained
7 Split Trained
8 Split Trained
9 Split Trained
10 Split Trained
Total Mean Score Obtained : 0.5389573339467848


In [ ]:
# # POWER ENSEMBLE

# #Here the cross validation by using the StratifiedKfold used to split the data into train,test and validation and repeatedly and Using the LGBMClassifier as Algorithm for building the model 
# skf =StratifiedKFold(n_splits=10, shuffle=True,random_state=2021) # for cross validation by StratifiedKfold
# scores = [] #list to store the sores obtained in training the model 
# preds2= [] #list to store the predictions 

# X = train_df.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
# y = train_df['Potability'] #[config_.preprocess.target[0]].astype(int)

# test_new = test_df.drop(columns=['region_area_']) #config_.preprocess.to_drop)
# test_fields = test_df['region_area_'] #[config_.preprocess.to_drop[0]]  #field

# #creating a for loop for the stratified k fold to perform the cross validation 
# i = 1
# for train, test in skf.split(X, y):
#     x_train, x_test= X.iloc[train], X.iloc[test] #spliting the data for x 
#     y_train, y_test = y.iloc[train],y.iloc[test] #spliting the data for y 
#     model = lgb.LGBMClassifier(learning_rate=0.08, n_estimators = 96,cat_smooth=10,metrics='binary_error',
#                                scale_pos_weight= 3.6,max_depth=16, seed =2021,
#                                num_leaves=49,reg_lambda=0.3) #Creating the model and tuning it with the different parameter 
#     model.fit(x_train, y_train)# fitting  the model on train data
#     print(i,'Split Trained') #print the splits that trained 
#     score = f1_score(y_test,model.predict(x_test)) # checking the accuracy of the model 
#     pred = model.predict_proba(test_new)[:,1] # making prediction
#     scores.append(score)
#     preds2.append(pred)
#     i += 1
# print('Total Mean Score Obtained :',np.mean(scores))

# preds_mean = (np.mean(preds2, axis=0)**4 + np.mean(preds ,axis=0) **4 )/2 # mean of all the predictions
# final = [] #list to store the final results 
# for x in preds_mean:
#     if x >= 0.51:
#         final.append(1)
#     else:
#         final.append(0)

# submit = test_df.copy() #Copy the submission file to the variable submit 
# submit['Potability'] = final
# submit[['region_area_','Potability']].to_csv('001xviibaseline3.csv',index=False) #creating and naming the file ready for submission

## 001 SUBS

In [ ]:
#Here the cross validation by using the StratifiedKfold used to split the data into train,test and validation and repeatedly and Using the LGBMClassifier as Algorithm for building the model 
skf =StratifiedKFold(n_splits=10, shuffle=True,random_state=2021) # for cross validation by StratifiedKfold
scores = [] #list to store the sores obtained in training the model 
preds= [] #list to store the predictions 

X = train_df.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
y = train_df['Potability'] #[config_.preprocess.target[0]].astype(int)

test_new = test_df.drop(columns=['region_area_']) #config_.preprocess.to_drop)
test_fields = test_df['region_area_'] #[config_.preprocess.to_drop[0]]  #field

#creating a for loop for the stratified k fold to perform the cross validation 
i = 1
for train, test in skf.split(X, y):
    x_train, x_test= X.iloc[train], X.iloc[test] #spliting the data for x 
    y_train, y_test = y.iloc[train],y.iloc[test] #spliting the data for y 
    model = lgb.LGBMClassifier(learning_rate=0.08, n_estimators = 96,cat_smooth=10,metrics='binary_error',
                               scale_pos_weight= 10.6 ,max_depth=16, seed =2021,
                               num_leaves=49,reg_lambda=0.3) #Creating the model and tuning it with the different parameter 
    model.fit(x_train, y_train)# fitting  the model on train data
    print(i,'Split Trained') #print the splits that trained 
    score = f1_score(y_test,model.predict(x_test)) # checking the accuracy of the model 
    pred = model.predict_proba(test_new)[:,1] # making prediction
    scores.append(score)
    preds.append(pred)
    i += 1
print('Total Mean Score Obtained :',np.mean(scores))

preds_mean = np.mean(preds, axis=0) # mean of all the predictions
final = [] #list to store the final results 
for x in preds_mean:
    if x >= 0.51:
        final.append(1)
    else:
        final.append(0)

submit = test_df.copy() #Copy the submission file to the variable submit 
submit['Potability'] = final
submit[['region_area_','Potability']].to_csv('001xviibaseline4.csv',index=False) #creating and naming the file ready for submission

1 Split Trained
2 Split Trained
3 Split Trained
4 Split Trained
5 Split Trained
6 Split Trained
7 Split Trained
8 Split Trained
9 Split Trained
10 Split Trained
Total Mean Score Obtained : 0.5808120490570688


In [ ]:
#Here the cross validation by using the StratifiedKfold used to split the data into train,test and validation and repeatedly and Using the LGBMClassifier as Algorithm for building the model 
skf =StratifiedKFold(n_splits=10, shuffle=True,random_state=2021) # for cross validation by StratifiedKfold
scores = [] #list to store the sores obtained in training the model 
preds= [] #list to store the predictions 

X = train_df.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
y = train_df['Potability'] #[config_.preprocess.target[0]].astype(int)

test_new = test_df.drop(columns=['region_area_']) #config_.preprocess.to_drop)
test_fields = test_df['region_area_'] #[config_.preprocess.to_drop[0]]  #field

#creating a for loop for the stratified k fold to perform the cross validation 
i = 1
for train, test in skf.split(X, y):
    x_train, x_test= X.iloc[train], X.iloc[test] #spliting the data for x 
    y_train, y_test = y.iloc[train],y.iloc[test] #spliting the data for y 
    model = lgb.LGBMClassifier(learning_rate=0.08, n_estimators = 96,cat_smooth=10,metrics='binary_error',
                               scale_pos_weight= 10.6 ,max_depth=16, seed =2021,
                               num_leaves=21,reg_lambda=0.3,) #Creating the model and tuning it with the different parameter 
    model.fit(x_train, y_train)# fitting  the model on train data
    print(i,'Split Trained') #print the splits that trained 
    score = f1_score(y_test,model.predict(x_test)) # checking the accuracy of the model 
    pred = model.predict_proba(test_new)[:,1] # making prediction
    scores.append(score)
    preds.append(pred)
    i += 1
print('Total Mean Score Obtained :',np.mean(scores))

preds_mean = np.mean(preds, axis=0) # mean of all the predictions
final = [] #list to store the final results 
for x in preds_mean:
    if x >= 0.51:
        final.append(1)
    else:
        final.append(0)

submit = test_df.copy() #Copy the submission file to the variable submit 
submit['Potability'] = final
submit[['region_area_','Potability']].to_csv('001xviibaseline5.csv',index=False) #creating and naming the file ready for submission

1 Split Trained
2 Split Trained
3 Split Trained
4 Split Trained
5 Split Trained
6 Split Trained
7 Split Trained
8 Split Trained
9 Split Trained
10 Split Trained
Total Mean Score Obtained : 0.5818348138833179


In [ ]:
#Here the cross validation by using the StratifiedKfold used to split the data into train,test and validation and repeatedly and Using the LGBMClassifier as Algorithm for building the model 
skf =StratifiedKFold(n_splits=10, shuffle=True,random_state=2021) # for cross validation by StratifiedKfold
scores = [] #list to store the sores obtained in training the model 
preds= [] #list to store the predictions 

X = train_df.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
y = train_df['Potability'] #[config_.preprocess.target[0]].astype(int)

test_new = test_df.drop(columns=['region_area_']) #config_.preprocess.to_drop)
test_fields = test_df['region_area_'] #[config_.preprocess.to_drop[0]]  #field

#creating a for loop for the stratified k fold to perform the cross validation 
i = 1
for train, test in skf.split(X, y):
    x_train, x_test= X.iloc[train], X.iloc[test] #spliting the data for x 
    y_train, y_test = y.iloc[train],y.iloc[test] #spliting the data for y 
    model = lgb.LGBMClassifier(learning_rate=0.08, n_estimators = 96,cat_smooth=10,metrics='binary_error',
                               scale_pos_weight= 11.6 ,max_depth=16, seed =2021,
                               num_leaves=21,reg_lambda=0.3,) #Creating the model and tuning it with the different parameter 
    model.fit(x_train, y_train)# fitting  the model on train data
    print(i,'Split Trained') #print the splits that trained 
    score = f1_score(y_test,model.predict(x_test)) # checking the accuracy of the model 
    pred = model.predict_proba(test_new)[:,1] # making prediction
    scores.append(score)
    preds.append(pred)
    i += 1
print('Total Mean Score Obtained :',np.mean(scores))

preds_mean = np.mean(preds, axis=0) # mean of all the predictions
final = [] #list to store the final results 
for x in preds_mean:
    if x >= 0.51:
        final.append(1)
    else:
        final.append(0)

submit = test_df.copy() #Copy the submission file to the variable submit 
submit['Potability'] = final
submit[['region_area_','Potability']].to_csv('001xviibaseline6.csv',index=False) #creating and naming the file ready for submission

1 Split Trained
2 Split Trained
3 Split Trained
4 Split Trained
5 Split Trained
6 Split Trained
7 Split Trained
8 Split Trained
9 Split Trained
10 Split Trained
Total Mean Score Obtained : 0.5831910378244277


# SUBS 3

In [26]:
#Here the cross validation by using the StratifiedKfold used to split the data into train,test and validation and repeatedly and Using the LGBMClassifier as Algorithm for building the model 
skf =StratifiedKFold(n_splits=10, shuffle=True,random_state=2021) # for cross validation by StratifiedKfold
scores = [] #list to store the sores obtained in training the model 
preds= [] #list to store the predictions 

X = train_df.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
y = train_df['Potability'] #[config_.preprocess.target[0]].astype(int)

test_new = test_df.drop(columns=['region_area_']) #config_.preprocess.to_drop)
test_fields = test_df['region_area_'] #[config_.preprocess.to_drop[0]]  #field

#creating a for loop for the stratified k fold to perform the cross validation 
i = 1
for train, test in skf.split(X, y):
    x_train, x_test= X.iloc[train], X.iloc[test] #spliting the data for x 
    y_train, y_test = y.iloc[train],y.iloc[test] #spliting the data for y 
    model = lgb.LGBMClassifier(learning_rate=0.08, n_estimators = 96,cat_smooth=10,metrics='binary_error',
                               scale_pos_weight= 10.6 ,max_depth=16, seed =2021,
                               num_leaves=21,reg_lambda=0.3,) #Creating the model and tuning it with the different parameter 
    model.fit(x_train, y_train)# fitting  the model on train data
    print(i,'Split Trained') #print the splits that trained 
    score = f1_score(y_test,model.predict(x_test)) # checking the accuracy of the model 
    pred = model.predict_proba(test_new)[:,1] # making prediction
    scores.append(score)
    preds.append(pred)
    i += 1
print('Total Mean Score Obtained :',np.mean(scores))

preds_mean = np.mean(preds, axis=0) # mean of all the predictions
final = [] #list to store the final results 
for x in preds_mean:
    if x >= 0.51:
        final.append(1)
    else:
        final.append(0)

submit = test_df.copy() #Copy the submission file to the variable submit 
submit['Potability'] = final
submit[['region_area_','Potability']].to_csv('001xviibaseline5.csv',index=False) #creating and naming the file ready for submission

1 Split Trained
2 Split Trained
3 Split Trained
4 Split Trained
5 Split Trained
6 Split Trained
7 Split Trained
8 Split Trained
9 Split Trained
10 Split Trained
Total Mean Score Obtained : 0.5818348138833179


In [33]:
#Here the cross validation by using the StratifiedKfold used to split the data into train,test and validation and repeatedly and Using the LGBMClassifier as Algorithm for building the model 
skf =StratifiedKFold(n_splits=10, shuffle=True,random_state=2021) # for cross validation by StratifiedKfold
scores = [] #list to store the sores obtained in training the model 
preds= [] #list to store the predictions 

X = train_df.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
y = train_df['Potability'] #[config_.preprocess.target[0]].astype(int)

test_new = test_df.drop(columns=['region_area_']) #config_.preprocess.to_drop)
test_fields = test_df['region_area_'] #[config_.preprocess.to_drop[0]]  #field

#creating a for loop for the stratified k fold to perform the cross validation 
i = 1
for train, test in skf.split(X, y):
    x_train, x_test= X.iloc[train], X.iloc[test] #spliting the data for x 
    y_train, y_test = y.iloc[train],y.iloc[test] #spliting the data for y 
    model = lgb.LGBMClassifier(learning_rate=0.08, n_estimators = 96,cat_smooth=10,metrics='binary_error',
                              #  neg_bagging_fraction=0.8,pos_bagging_fraction=0.7,
                               scale_pos_weight= 10.6 ,max_depth=16, seed =2021,
                               num_leaves=49,reg_lambda=0.3) #Creating the model and tuning it with the different parameter 
    model.fit(x_train, y_train)# fitting  the model on train data
    print(i,'Split Trained') #print the splits that trained 
    score = f1_score(y_test,model.predict(x_test)) # checking the accuracy of the model 
    pred = model.predict_proba(test_new)[:,1] # making prediction
    scores.append(score)
    preds.append(pred)
    i += 1
print('Total Mean Score Obtained :',np.mean(scores))

preds_mean = np.mean(preds, axis=0) # mean of all the predictions
final = [] #list to store the final results 
for x in preds_mean:
    if x >= 0.51:
        final.append(1)
    else:
        final.append(0)

# submit = test_df.copy() #Copy the submission file to the variable submit 
# submit['Potability'] = final
# submit[['region_area_','Potability']].to_csv('003xviibaseline4.csv',index=False) #creating and naming the file ready for submission

1 Split Trained
2 Split Trained
3 Split Trained
4 Split Trained
5 Split Trained
6 Split Trained
7 Split Trained
8 Split Trained
9 Split Trained
10 Split Trained
Total Mean Score Obtained : 0.5808120490570688


In [35]:
test_df['Potability'] = final

new_train = pd.concat([train_df,test_df]).reset_index(drop=True)


In [37]:
#Here the cross validation by using the StratifiedKfold used to split the data into train,test and validation and repeatedly and Using the LGBMClassifier as Algorithm for building the model 
skf =StratifiedKFold(n_splits=10, shuffle=True,random_state=2021) # for cross validation by StratifiedKfold
scores = [] #list to store the sores obtained in training the model 
preds= [] #list to store the predictions 

X = new_train.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
y = new_train['Potability'] #[config_.preprocess.target[0]].astype(int)

test_new = test_df.drop(columns=['region_area_', 'Potability']) #config_.preprocess.to_drop)
test_fields = test_df['region_area_'] #[config_.preprocess.to_drop[0]]  #field

#creating a for loop for the stratified k fold to perform the cross validation 
i = 1
for train, test in skf.split(X, y):
    x_train, x_test= X.iloc[train], X.iloc[test] #spliting the data for x 
    y_train, y_test = y.iloc[train],y.iloc[test] #spliting the data for y 
    model = lgb.LGBMClassifier(learning_rate=0.08, n_estimators = 96,cat_smooth=10,metrics='binary_error',
                              #  neg_bagging_fraction=0.8,pos_bagging_fraction=0.7,
                               scale_pos_weight= 10.6 ,max_depth=16, seed =2021,
                               num_leaves=49,reg_lambda=0.3, reg_alpha=0.3) #Creating the model and tuning it with the different parameter 
    model.fit(x_train, y_train)# fitting  the model on train data
    print(i,'Split Trained') #print the splits that trained 
    score = f1_score(y_test,model.predict(x_test)) # checking the accuracy of the model 
    pred = model.predict_proba(test_new)[:,1] # making prediction
    scores.append(score)
    preds.append(pred)
    i += 1
print('Total Mean Score Obtained :',np.mean(scores))

preds_mean = np.mean(preds, axis=0) # mean of all the predictions
final = [] #list to store the final results 
for x in preds_mean:
    if x >= 0.51:
        final.append(1)
    else:
        final.append(0)

submit = test_df.copy() #Copy the submission file to the variable submit 
submit['Potability'] = final
submit[['region_area_','Potability']].to_csv('003xviibaseline7.csv',index=False) #creating and naming the file ready for submission

1 Split Trained
2 Split Trained
3 Split Trained
4 Split Trained
5 Split Trained
6 Split Trained
7 Split Trained
8 Split Trained
9 Split Trained
10 Split Trained
Total Mean Score Obtained : 0.7032938196522149
